In [1]:
from ngsolve import *

In [2]:
# from netgen.csg import Pnt,SplineCurve2d,CSGeometry,Revolution,Sphere
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.comp import IntegrationRuleSpaceSurface

In [3]:
from scipy.spatial import cKDTree

In [4]:
from es_utils import pos_transformer, SurfacehInterp

In [5]:
from geometry import *

In [6]:
from esfem import WillMoreMDR

In [7]:
from ngsolve.webgui import Draw

In [8]:
from ngsolve import Mesh

In [9]:
import sympy as sp
a,b,c = sp.symbols('x y z')

## 点到参数曲面的投影：整体思路（k-d tree 初值 + 向量化 Newton 迭代）

设参数曲面为
\begin{align*}
s(\phi,\psi)\in\mathbb{R}^3,
\end{align*}
给定一批查询点
\begin{align*}
q_i \in \mathbb{R}^3,\quad i=1,\dots,n,
\end{align*}
目标是为每个点找到参数 $(\phi_i,\psi_i)$，使得曲面点 $s(\phi_i,\psi_i)$ 到 $q_i$ 的距离最小，即求解投影问题
\begin{align*}
(\phi_i,\psi_i) = \arg\min_{\phi,\psi}\; f_i(\phi,\psi),
\qquad
f_i(\phi,\psi)=\|s(\phi,\psi)-q_i\|^2.
\end{align*}

---

### 1. 用 k-d tree 给出初始猜测

先在预先采样的曲面点集合中，对每个查询点 \(q_i\) 找到最近邻采样点的索引（k-d tree 查询）。将该采样点对应的参数值作为 Newton 迭代初值：
\begin{align*}
(\phi_i^{(0)},\psi_i^{(0)}) \approx \text{nearest-sample-parameters}(q_i).
\end{align*}
这样做的目的是：让初值尽量落在“正确的 basin”里，提高 Newton 法的收敛概率与速度。

---

### 2. 用 Newton 法最小化距离平方（向量化批量迭代）

对每个点 $i$，在第 $k$ 次迭代，用当前参数 $(\phi_i^{(k)},\psi_i^{(k)})$ 计算：

**(a) 曲面点与残差：**
\begin{align*}
s_i^{(k)} &= s(\phi_i^{(k)},\psi_i^{(k)}),\\
r_i^{(k)} &= s_i^{(k)} - q_i.
\end{align*}

**(b) 雅可比矩阵（对参数的导数）：**
\begin{align*}
J_i^{(k)} =
\begin{bmatrix}
\frac{\partial s}{\partial \phi}(\phi_i^{(k)},\psi_i^{(k)}) &
\frac{\partial s}{\partial \psi}(\phi_i^{(k)},\psi_i^{(k)})
\end{bmatrix}
\in \mathbb{R}^{3\times 2}.
\end{align*}

**(c) 目标函数梯度：**
对
\begin{align*}
f_i(\phi,\psi)=\|s(\phi,\psi)-q_i\|^2 = r_i^\top r_i,
\end{align*}
有
\begin{align*}
\nabla f_i(\phi,\psi) = 2\,J_i^\top r_i \in \mathbb{R}^{2}.
\end{align*}

**(d) Hessian 与 Newton 步长：**
用你实现/符号推导得到的 Hessian（或近似 Hessian）
\begin{align*}
H_i^{(k)} \approx \nabla^2 f_i(\phi_i^{(k)},\psi_i^{(k)}) \in \mathbb{R}^{2\times 2},
\end{align*}
解线性方程得到 Newton 更新量：
\begin{align*}
H_i^{(k)} \,\Delta_i^{(k)} = -\nabla f_i^{(k)},
\qquad
\Delta_i^{(k)}=
\begin{bmatrix}
\Delta\phi_i^{(k)}\\
\Delta\psi_i^{(k)}
\end{bmatrix}.
\end{align*}

**(e) 参数更新与截断（clamp）：**
\begin{align*}
\phi_i^{(k+1)} &= \phi_i^{(k)} + \Delta\phi_i^{(k)},\\
\psi_i^{(k+1)} &= \psi_i^{(k)} + \Delta\psi_i^{(k)},
\end{align*}
并将参数限制在允许范围（例如 $[\phi_{\min},\phi_{\max}]$、$[\psi_{\min},\psi_{\max}]$）：
\begin{align*}
\phi_i^{(k+1)} \leftarrow \mathrm{clip}\bigl(\phi_i^{(k+1)},\phi_{\min},\phi_{\max}\bigr),\qquad
\psi_i^{(k+1)} \leftarrow \mathrm{clip}\bigl(\psi_i^{(k+1)},\psi_{\min},\psi_{\max}\bigr).
\end{align*}

---

### 3. 收敛判据与异常点记录

常用的收敛判据是更新量的范数足够小：
\begin{align*}
\|\Delta_i^{(k)}\|_2 < \text{tol}.
\end{align*}
达到阈值则认为第 \(i\) 个点收敛；否则继续迭代，最多迭代 `max_iter` 次。

迭代结束后，计算最终投影点：
\begin{align*}
p_i = s(\phi_i,\psi_i),
\end{align*}
并可记录未收敛点的索引及其最后一次更新量范数，用于调参或定位数值问题。

In [10]:
class ParamSurface:
    '''
        Parametric surface of revolution, defined by a profile curve r(phi), z(phi).
        The surface is given by:
            S(theta, phi) = ( r(phi)*cos(theta), r(phi)*sin(theta), z(phi) )
        where theta in [0, 2*pi), phi in [phi_min, phi_max].
        The profile curve is assumed to be given in a parametric form:
            r = r(phi), z = z(phi)
        with phi in [phi_min, phi_max].
        
        param: list of 3 symbolic expressions [x(phi, psi), y(phi, psi), z(phi, psi)]
    '''
    def __init__(self, params, phi_lim=(0,2*np.pi), psi_lim=(0,2*np.pi),num_points=10000):
        # Param: 2d list of symbolic expression
        self.x_param, self.y_param, self.z_param = params
        self.phi_lim = phi_lim
        self.psi_lim = psi_lim
        func = sym.lambdify((phi, psi), params)
        self.param_np = lambda x1,y1: np.squeeze(func(x1,y1)).T
        # Jacobian 矩阵 (3x2): 第一行是对phi求导，第二行是对psi求导
        self.jacobian = sym.Matrix([[f.diff(var) for var in (phi, psi)] for f in params])
        self.jacobian_func = sym.lambdify((phi, psi), self.jacobian)
        
        # 定义目标函数 f = (x-a)^2 + (y-b)^2 + (z-c)^2，其中a,b,c是符号变量
        f = (self.x_param - a)**2 + (self.y_param - b)**2 + (self.z_param - c)**2
        # Hessian 矩阵 (2x2)
        self.hessian = sym.hessian(f, [phi, psi])
        self.hessian_func = sym.lambdify((phi, psi, a, b, c), self.hessian, 'numpy')
        self.build_tree(num_points)
    
    def build_tree(self,num_points):
        self.phi_samples, self.psi_samples, self.points_3d = self.sample_points_on_surface(num_points)
        self.tree = cKDTree(self.points_3d)
        
    def proj_param(self, query_points, max_iter=10, tol=1e-6):
        """
        Project query points onto the parametric surface using k-d tree for initial guesses
        and vectorized Newton's method for refinement.
        
        Args:
            query_points: np.ndarray of shape (n, 3), query points in 3D space
            max_iter: Maximum number of Newton iterations
            tol: Tolerance for convergence (norm of parameter update)
        
        Returns:
            tuple: (phi_values, psi_values), arrays of shape (n,) with projected parameters
        """
        # Get initial guesses from k-d tree
        distances, indices = self.tree.query(query_points, k=1)
        phi_values = self.phi_samples[indices].copy()
        psi_values = self.psi_samples[indices].copy()

        # Track which points have converged
        n_points = query_points.shape[0]
        converged = np.zeros(n_points, dtype=bool)
        
        # Newton iterations
        for _ in range(max_iter):
            # Skip converged points
            if converged.all():
                break
                
            # Evaluate surface points s(phi, psi)
            s = self.param_np(phi_values, psi_values)  # Shape (n, 3)

            # Compute residuals (s - q)
            residuals = s - query_points  # Shape (n, 3)

            # Compute Jacobians for all points, ijk: k index; i: axis; j: derivative
            J = np.array(self.jacobian_func(phi_values, psi_values))  # Shape (n, 3, 2)

            # Compute gradient: 2 * J^T @ residual grad of the distance
            grad_f = 2 * np.einsum('jik,kj->ki', J, residuals)  # Shape (n, 2)

            hessian_f = self.hessian_func(phi_values,psi_values,query_points[:,0],
                                         query_points[:,1],query_points[:,2])

            hessian_f = np.transpose(hessian_f, (2, 0, 1))  # 形状 (4000, 2, 2)

            det_H = np.linalg.det(hessian_f) # 形状 (4000,)

            valid = np.abs(det_H) > 1e-6
            # Initialize updates
            delta = np.zeros((n_points, 2))

            # Compute Newton step for valid points
            if valid.any():
                # Solve H @ delta = -grad_f for valid points
                delta[valid] = np.linalg.solve(hessian_f[valid], -grad_f[valid])

            # Update parameters where not converged
            update_mask = (~converged) & (np.linalg.norm(delta, axis=1) > tol)
            phi_values[update_mask] += delta[update_mask, 0]
            psi_values[update_mask] += delta[update_mask, 1]

            # Clamp parameters to valid ranges
            phi_values = np.clip(phi_values, self.phi_lim[0], self.phi_lim[1])
            psi_values = np.clip(psi_values, self.psi_lim[0], self.psi_lim[1])

            # Update convergence status
            converged[update_mask] = np.linalg.norm(delta[update_mask], axis=1) < tol

        # 计算最终曲面点
        points = self.param_np(phi_values, psi_values)

        if not max_iter == 0:
            # 记录未收敛点的容差信息
            tol_info = {
                'non_converged_indices': np.where(~converged)[0],
                'final_delta_norms': np.linalg.norm(delta[~converged], axis=1)
            }
        else:
            tol_info = {}
        return phi_values, psi_values, points, tol_info
    
    
    def get_coarse_param(self, query_points):
        distances, indices = self.tree.query(query_points, k=1) 
        return distances, indices
    
    def sample_points_on_surface(self, n_points):
        """
        根据 Jacobian 面积元素在参数平面上加权采样点，并映射到曲面上。
        输入:
            n_points: 采样点总数
            phi_range: (phi_min, phi_max)
            psi_range: (psi_min, psi_max)
        
        输出:
            phi_samples, psi_samples: 参数平面上的采样
            points_3d: 映射到曲面上的 3D 坐标
        """
        # 1. 在参数平面上生成粗网格
        N_grid = int(np.sqrt(5 * n_points))  # 粗略，网格比点数多几个
        phi_lin = np.linspace(self.phi_lim[0], self.phi_lim[1], N_grid)
        psi_lin = np.linspace(self.psi_lim[0], self.psi_lim[1], N_grid)
        Phi, Psi = np.meshgrid(phi_lin, psi_lin, indexing='ij')
        Phi_flat = Phi.ravel()
        Psi_flat = Psi.ravel()

        # 2. 计算每个网格点的面积元素
        J_vals = self.jacobian_func(Phi_flat, Psi_flat)   # shape (N, 3, 2) 或 (N,6)展开的情况
        # 如果 lambdify 返回 (N,6) 扁平展开，需要 reshape
        if J_vals.ndim == 2 and J_vals.shape[1] == 6:
            J_vals = J_vals.reshape(-1,3,2)
        J_vals = np.moveaxis(J_vals, 2, 0) 
        
        # 面积元素
        self.dS = np.sqrt(np.linalg.det(np.matmul(np.transpose(J_vals, (0,2,1)), J_vals)))  
        # sqrt(det(J^T J)), shape (N,)
        # 3. 面积加权采样（用CDF逆变换）
        eps = 1e-16
        weights = self.dS / np.sum(self.dS+eps)
        cdf = np.cumsum(weights)
        
        # 从均匀 [0,1] 采样
        u = np.random.rand(n_points)
        indices = np.searchsorted(cdf, u)
        
        phi_samples = Phi_flat[indices]
        psi_samples = Psi_flat[indices]
        
        points_3d = self.param_np(phi_samples, psi_samples)  # 或用你对应的曲面函数
        return phi_samples, psi_samples, points_3d

In [11]:
from geometry.param_curve import phi, psi
import sympy as sp
X_perturbed = sp.Matrix([
    (1 + 0.65*sp.cos(phi)) * sp.cos(psi),
    (1 + 0.65*sp.cos(phi)) * sp.sin(psi),
    0.65*sp.sin(phi) + 0.3*sp.sin(5*psi)
])

### 测试
* 根据参数表达式 X_perturbed 来生成ParamSurface对象，预采样的点数是 num_points
* 给出一些点（需要被投影的点） points_3d
* phi_values, psi_values, points, tol_info = obj.proj_param(points_3d) 将这些点投影到曲面上
* 

In [12]:
obj = ParamSurface(X_perturbed,num_points=400000)

In [13]:
points_3d = obj.points_3d*0.99

In [14]:
phi_values, psi_values, points, tol_info = obj.proj_param(points_3d)

In [15]:
print("Residue in optimation is {}".format(np.max(tol_info['final_delta_norms'])))
print("Max projected distance is {}".format(max(np.linalg.norm(points-points_3d,axis = 1))))

Residue in optimation is 0.005404827041991507
Max projected distance is 0.016940283439217604


In [16]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,5))
plt.scatter(phi_samples, psi_samples, s=10, c='blue', alpha=0.6)
plt.xlabel(r'$\phi$')
plt.ylabel(r'$\psi$')
plt.title('Weighted parameter plane sampling')
plt.grid(True)
plt.show()

NameError: name 'phi_samples' is not defined

<Figure size 600x500 with 0 Axes>

In [189]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [190]:
points_3d = points 

In [191]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe"  # 生成独立 iframe，稳定显示
# pio.renderers.default = "colab" # 如果在 Colab 中

fig = go.Figure(data=[go.Scatter3d(
    x=points_3d[:,0],
    y=points_3d[:,1],
    z=points_3d[:,2],
    mode='markers',
    marker=dict(size=3, color=points_3d[:,2], colorscale='Viridis')
)])
fig.show()


这样使用plotly就可以可视化看到采样点的分布